In [1]:
import torch
torch.cuda.empty_cache()

### 2. Load Base Model

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "NousResearch/Yarn-Mistral-7b-64k"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, use_flash_attention_2=True, quantization_config=bnb_config, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Try Base Model

In [3]:
from IPython.display import display, Markdown

def make_inference(instruction, context = None):
  if context:
    prompt = f"Below is an instruction that describes a task, paired with an input that provides further context.\n\n### Instruction: \n{instruction}\n\n### Input: \n{context}\n\n### Response: \n"
  else:
    prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: \n{instruction}\n\n### Response: \n"
  inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
  outputs = model.generate(**inputs, max_new_tokens=150)
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))

make_inference("What scripture did Luther use to back up his opinion that churchmen were not infallible?", "In January 1519, at Altenburg in Saxony, the papal nuncio Karl von Miltitz adopted a more conciliatory approach. Luther made certain concessions to the Saxon, who was a relative of the Elector, and promised to remain silent if his opponents did. The theologian Johann Eck, however, was determined to expose Luther's doctrine in a public forum. In June and July 1519, he staged a disputation with Luther's colleague Andreas Karlstadt at Leipzig and invited Luther to speak. Luther's boldest assertion in the debate was that Matthew 16:18 does not confer on popes the exclusive right to interpret scripture, and that therefore neither popes nor church councils were infallible. For this, Eck branded Luther a new Jan Hus, referring to the Czech reformer and heretic burned at the stake in 1415. From that moment, he devoted himself to Luther's defeat.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2023-11-23 04:17:46.597219: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-23 04:17:46.741747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 04:17:46.741800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 04:17:46.742207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 04:17:46.806264: I tensorflow/core/platform/cpu_feature_g

Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
What scripture did Luther use to back up his opinion that churchmen were not infallible?

### Input: 
In January 1519, at Altenburg in Saxony, the papal nuncio Karl von Miltitz adopted a more conciliatory approach. Luther made certain concessions to the Saxon, who was a relative of the Elector, and promised to remain silent if his opponents did. The theologian Johann Eck, however, was determined to expose Luther's doctrine in a public forum. In June and July 1519, he staged a disputation with Luther's colleague Andreas Karlstadt at Leipzig and invited Luther to speak. Luther's boldest assertion in the debate was that Matthew 16:18 does not confer on popes the exclusive right to interpret scripture, and that therefore neither popes nor church councils were infallible. For this, Eck branded Luther a new Jan Hus, referring to the Czech reformer and heretic burned at the stake in 1415. From that moment, he devoted himself to Luther's defeat.

### Response: 
Matthew 16:18

### Explanation: 
The scripture that Luther used to back up his opinion that churchmen were not infallible was Matthew 16:18, which states, "And I tell you, you are Peter, and on this rock I will build my church, and the gates of hell shall not prevail against it." This verse was often used by the Catholic Church to support the doctrine of papal infallibility, which states that the pope is free from error when speaking on matters of faith and morals. Luther argued that this verse did not confer on popes the exclusive right to interpret scripture, and that therefore neither popes nor church counc

### 7. Inference

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "NousResearch/Yarn-Mistral-7b-128k"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/home/cs01/miniconda3/envs/qlora/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "YM64k-Fine-Tune-YM64k/checkpoint-1000")

In [5]:
from IPython.display import display, Markdown

def make_inference_(instruction, context = None):
  if context:
    prompt = f"Below is an instruction that describes a task, paired with an input that provides further context.\n\n### Instruction: \n{instruction}\n\n### Input: \n{context}\n\n### Response: \n"
  else:
    prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: \n{instruction}\n\n### Response: \n"
  inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
  outputs = model.generate(**inputs, max_new_tokens=150)
  display(Markdown((tokenizer.decode(outputs[0], skip_special_tokens=True))))

make_inference_("What scripture did Luther use to back up his opinion that churchmen were not infallible?", "In January 1519, at Altenburg in Saxony, the papal nuncio Karl von Miltitz adopted a more conciliatory approach. Luther made certain concessions to the Saxon, who was a relative of the Elector, and promised to remain silent if his opponents did. The theologian Johann Eck, however, was determined to expose Luther's doctrine in a public forum. In June and July 1519, he staged a disputation with Luther's colleague Andreas Karlstadt at Leipzig and invited Luther to speak. Luther's boldest assertion in the debate was that Matthew 16:18 does not confer on popes the exclusive right to interpret scripture, and that therefore neither popes nor church councils were infallible. For this, Eck branded Luther a new Jan Hus, referring to the Czech reformer and heretic burned at the stake in 1415. From that moment, he devoted himself to Luther's defeat.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context.

### Instruction: 
What scripture did Luther use to back up his opinion that churchmen were not infallible?

### Input: 
In January 1519, at Altenburg in Saxony, the papal nuncio Karl von Miltitz adopted a more conciliatory approach. Luther made certain concessions to the Saxon, who was a relative of the Elector, and promised to remain silent if his opponents did. The theologian Johann Eck, however, was determined to expose Luther's doctrine in a public forum. In June and July 1519, he staged a disputation with Luther's colleague Andreas Karlstadt at Leipzig and invited Luther to speak. Luther's boldest assertion in the debate was that Matthew 16:18 does not confer on popes the exclusive right to interpret scripture, and that therefore neither popes nor church councils were infallible. For this, Eck branded Luther a new Jan Hus, referring to the Czech reformer and heretic burned at the stake in 1415. From that moment, he devoted himself to Luther's defeat.

### Response: 
The scripture that Luther used to back up his opinion that churchmen were not infallible was Matthew 16:18, which states: "And I tell you, you are Peter, and on this rock I will build my church, and the gates of Hades will not prevail against it." Luther interpreted this passage to mean that the church was not built on the authority of the pope, but rather on the foundation of Jesus Christ and the teachings of the Bible. He believed that the church should be led by those who were faithful to the teachings of the Bible, rather than by those who claimed infallibility for themselves. This interpretation was controversial at the time, and led to Luther's excommunication